In [1]:
import requests
import re
from bs4 import BeautifulSoup, Comment
import pandas as pd
import numpy as np
import string
import multiprocess as mp
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn.model_selection import GridSearchCV
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_bat = pd.read_csv('asset/batters.csv',index_col=0)
df_bat

FileNotFoundError: [Errno 2] No such file or directory: 'asset/batters.csv'

In [ ]:
def career_stats(df, force={}, skill='batter'):
    cols = list(df.select_dtypes('object').columns)
    cols.remove('Player')
    keep = [x for x in df.columns if x not in cols]
    df = df[keep]
    colMax = ['Age','Season','oWAR_3','oWAR_7','oWAR_10','oWAR_15', 'HOF', 'last_game', 'Year']
    colMean = ['BA','OBP','SLG','OPS','OPS+','BAbip','Pos_2','rOBA', 'Rbat+']
    mod = {}    
    for col in df.columns:
        if col == 'Player':
            pass
        elif '%' in col:
            mod[col]='mean'
        elif col in colMean:
            mod[col]='mean'
        elif col in colMax:
            mod[col]= 'max'
        else:
            mod[col]= 'sum'
    if force:
        for key, value in force.items():
            mod[key]= value
            
    df2 = df.groupby('Player').agg(mod).reset_index()
    if skill == 'batter':
        df2['Pos_mean'] = df2['Pos_adjust']/df2['G']
        
    df2['SLG'] = (df2['HR']*4 + df2['3B']*3 + df2['2B']*2 + (df2['H']-df2['HR']-df2['3B']-df2['2B']))/df2['AB']
    df2['BA'] = df2['H']/df2['AB']
    df2['OBP'] = (df2['H']+df2['BB'])/(df2['PA']-df2['SF'])
    df2['OPS'] = df2['OBP']+df2['SLG']
    
    return df2


def simScore(df, name = 'Henry Aaron', n=10):
    df['Bill Score'] = 0
    stats_p1 = df.loc[name]
    p2 = df.loc[df.index != name,:]
    df.loc[name,'Bill Score'] = 1000
    for i in range(len(p2)):
        stats_p2 = p2.iloc[i]
        player = stats_p2.name
        G = (stats_p1.G-stats_p2.G)/20
        AB = (stats_p1.AB-stats_p2.AB)/75
        R = (stats_p1.R-stats_p2.R)/10
        H = (stats_p1.H-stats_p2.H)/15
        Doubles = (stats_p1['2B']-stats_p2['2B'])/5
        Triples = (stats_p1['3B']-stats_p2['3B'])/4
        HR = (stats_p1.HR-stats_p2.HR)/2
        RBI = (stats_p1.RBI-stats_p2.RBI)/10
        BB = (stats_p1.BB-stats_p2.BB)/25
        SO = (stats_p1.SO-stats_p2.SO)/150
        SB = (stats_p1.SB-stats_p2.SB)/20
        BA = (stats_p1.BA-stats_p2.BA)/0.001
        SLG = (stats_p1.SLG-stats_p2.SLG)/0.002
        try:
            POS = (stats_p1.Pos_mean-stats_p2.Pos_mean)*12
        except:
            POS = 0
        simScore = 1000 - abs(POS) - abs(G)- abs(AB)- abs(R)- abs(H)- abs(Doubles)- abs(Triples)- abs(HR)- abs(RBI)- abs(BB)-abs(SO)- abs(SB)- abs(BA)- abs(SLG)
        df.loc[player,'Bill Score'] = simScore    
    df = df.sort_values('Bill Score', ascending=False)[:n]
    return df

In [ ]:
## we have individual season by season data of players... now we need to collected full career summary data

df = career_stats(df_bat)
df = df[['Player'] + ['Season','G', 'AB', 'R', 'H','2B','3B', 'HR','RBI', 'SB','BA', 'SLG','OPS', 'TB','Rbat','WAA','RAR','oRAR','RE24','Pos_mean','HOF','WAR','oWAR', 'Rbat+','Rfield','Rbaser','last_game']] #'BB', 'SO', 'SB'
df = df.merge(df_bat[['Player', 'Player_name']].drop_duplicates('Player'),how='left', on='Player')
df = df[df['last_game']<=2015]

df.reset_index(drop=True, inplace=True)
df.dropna(inplace=True)

## grabbing the HOF information for each player
link = 'https://www.baseball-reference.com/awards/hof.shtml'
HOF = pd.read_html(link)[0]
HOF = HOF[HOF['Inducted As']=='Player']
HOF.reset_index(drop=True, inplace=True)


df = df.merge(HOF[['Name', 'Voted By','Inducted As']], left_on='Player_name', right_on='Name', how='left')
df.loc[(df['Voted By'].isna())&(df['HOF']==1),'Voted By']='BBWAA'
df

In [3]:
df_hof = df[df['HOF']==1]
df_hof.reset_index(drop=True,inplace=True)
df_hof['Voted By'].value_counts()

NameError: name 'df' is not defined

In [4]:
df['Voted By'] = df['Voted By'].fillna('Not_hof')
df['Inducted As'] = df['Inducted As'].fillna('Not_hof')
keep = ['BBWAA','Not_hof']
keep2 = ['Player', 'Not_hof']
df = df[(df['Voted By'].isin(keep))&(df['Inducted As'].isin(keep2))]
df.reset_index(drop=True,inplace=True)
df.loc[df['Player']=='gwynnto01', 'Player_name'] = 'Tony Gwynn (M)'
df['HOF'].value_counts()

NameError: name 'df' is not defined

## Feature Engineering

https://www.mlb.com/glossary


In [5]:
def awards(player, award, n = None, df=df_bat):
    temp = df[df['Player']==player]
    try:        
        if award == 'AS' or award == 'GG':
            temp1 = temp[temp['Year']<=1933]
            if len(temp1)==0:
                add=0
            else:
                add = temp1['Awards'].str.contains('MVP',n).value_counts()[True]
            return temp['Awards'].str.contains(award).value_counts()[True] + add

        elif award == 'GG':
            return temp['Awards'].str.contains(award).value_counts()[True]
        elif award == 'MVP':
            if n:
                return temp['Awards'].str.contains('MVP-[1-{n}].',n).value_counts()[True]
            else:
                return temp['Awards'].str.contains('MVP',n).value_counts()[True]
    except:
        return 0

def rolling_max(player, n, col,df=df_bat):    
    temp = df[df['Player']==player]
    temp.reset_index(drop=True, inplace=True)    
    if sum(temp[col].rolling(n).sum().isna())== len(temp):
        if col == 'BA':            
            return sum(temp['H'])/sum(temp['AB'])
        elif col == 'SLG':            
            return (sum(temp['HR'])*4 + sum(temp['3B'])*3 + sum(temp['2B'])*2 + (sum(temp['H'])-sum(temp['HR'])-sum(temp['3B'])-sum(temp['2B'])))/sum(temp['AB'])
        else:
            return sum(temp[col])
    idx = temp[col].rolling(n).sum().idxmax()
    #print(player,idx)
    
    if col == 'BA':
        temp = temp.iloc[idx-n+1:idx+1]
        return sum(temp['H'])/sum(temp['AB'])
    elif col == 'SLG':
        temp = temp.iloc[idx-n+1:idx+1]
        return (sum(temp['HR'])*4 + sum(temp['3B'])*3 + sum(temp['2B'])*2 + (sum(temp['H'])-sum(temp['HR'])-sum(temp['3B'])-sum(temp['2B'])))/sum(temp['AB'])
    else:
        return max(temp[col].rolling(n).sum().dropna())

def col_max(player, col, df=df_bat):
    temp = df[df['Player']==player]
    return max(temp[col])

def year_max(player, col, percent=.9, df=df_bat):
    if col == 'BA':
        df1 = df[df['G']>100]
    else:
        df1 = df.copy()    
    yearmax = df1[[col, 'Year']].groupby('Year').agg('max').reset_index()
    yearmax.rename(columns={col:col+'_max'}, inplace=True)
    temp = df1[df1['Player']==player]
    if col == 'BA':
        temp = temp[temp['G']>40]
    temp = temp.merge(yearmax, on='Year', how='left')
    temp[col+'_new'] = np.where(temp[col]>=percent*temp[col+'_max'],1,0)
    return sum(temp[col+'_new'])


def bestof(player, n, col, df=df_bat):
    temp = df[df['Player']==player]
    temp.reset_index(drop=True, inplace=True)
    if len(temp)<=n:
        if col == 'BA':            
            return sum(temp['H'])/sum(temp['AB'])
        elif col == 'SLG':            
            return (sum(temp['HR'])*4 + sum(temp['3B'])*3 + sum(temp['2B'])*2 + (sum(temp['H'])-sum(temp['HR'])-sum(temp['3B'])-sum(temp['2B'])))/sum(temp['AB'])
        else:
            return sum(temp[col])
    else:
        if col == 'BA':
            temp = temp.nlargest(n, 'BA')
            return sum(temp['H'])/sum(temp['AB'])
        elif col == 'SLG':
            temp = temp.nlargest(n, 'SLG')
            return (sum(temp['HR'])*4 + sum(temp['3B'])*3 + sum(temp['2B'])*2 + (sum(temp['H'])-sum(temp['HR'])-sum(temp['3B'])-sum(temp['2B'])))/sum(temp['AB'])
        else:
            return sum(temp[col].nlargest(n))

NameError: name 'df_bat' is not defined

In [6]:
# %%time

# df['WAR_best_5']= df['Player'].apply(lambda x:bestof(x,5,'WAR'))
# df['WAR_best_7']= df['Player'].apply(lambda x:bestof(x,7,'WAR'))
# df['WAR_5']= df['Player'].apply(lambda x:rolling_max(x,5,'WAR'))
# df['WAR_7']= df['Player'].apply(lambda x:rolling_max(x,7,'WAR'))
# df['oWAR_5']= df['Player'].apply(lambda x:rolling_max(x,5,'oWAR'))
# df['oWAR_7']= df['Player'].apply(lambda x:rolling_max(x,7,'oWAR'))
# df['oWAR_best_5']= df['Player'].apply(lambda x:bestof(x,5,'oWAR'))
# df['oWAR_best_7']= df['Player'].apply(lambda x:bestof(x,7,'oWAR'))
# df['RAR_best_5']= df['Player'].apply(lambda x:bestof(x,5,'RAR'))
# df['RAR_best_7']= df['Player'].apply(lambda x:bestof(x,7,'RAR'))
# df['AS'] = df['Player'].apply(lambda x:awards(x,'AS'))
# df['MVP'] = df['Player'].apply(lambda x:awards(x,'MVP', n=10))
# df['RAR_max'] = df['Player'].apply(lambda x:col_max(x,'RAR'))
# df['H_ymax'] = df['Player'].apply(lambda x:year_max(x,'H'))
# df['HR_ymax'] = df['Player'].apply(lambda x:year_max(x,'HR'))
# df['R_ymax'] = df['Player'].apply(lambda x:year_max(x,'R'))
# df['RBI_ymax'] = df['Player'].apply(lambda x:year_max(x,'RBI'))
# df['BA_ymax'] = df['Player'].apply(lambda x:year_max(x,'BA'))
# df['ten'] = np.where((df['Season']>=10),1,0)

df = pd.read_csv('asset/dfFeaturesAdded.csv', index_col=0)
df

FileNotFoundError: [Errno 2] No such file or directory: 'asset/dfFeaturesAdded.csv'

In [7]:
## at this point we can play around with all of the newly created features and iterate through the model creation
## to see what accuracy the model predicts. We performed these activities and ultimately settled on the following

#hof_final = df.drop(columns=['Name', 'Voted By', 'Inducted As', 'Pos_mean', 'G', 'AB','Player_name', 'last_game'])
#hof_final.set_index('Player', inplace=True)

hof_final = pd.read_csv('asset/hof_final.csv', index_col=0)

hof_final.dropna(inplace=True)
hof_final = hof_final[['H_ymax', 'HR_ymax', 'R_ymax', 'RBI_ymax', 'BA_ymax','RAR','WAR',
                       'oWAR','WAR_best_7','WAR_7','oWAR_7','oWAR_best_7','RAR_best_7',
                       'RAR_max','AS', 'MVP', 'Rfield', 'HOF']]
hof_final

FileNotFoundError: [Errno 2] No such file or directory: 'asset/hof_final.csv'

In [8]:
hof_final_train, hof_final_test = train_test_split(hof_final,stratify=hof_final[['HOF']], test_size=0.2, random_state=0)
hof_final_train

NameError: name 'hof_final' is not defined

In [9]:
hof_final_train['HOF'].value_counts()

NameError: name 'hof_final_train' is not defined

In [10]:
hof_final_test['HOF'].value_counts()

NameError: name 'hof_final_test' is not defined

In [11]:
fold = 10
skf = StratifiedKFold(n_splits=fold, shuffle=False)
ntrees = 200
C = 10
gamma = 1
models = {'SVM': SVC(C=1, gamma='scale',probability=True, kernel = 'rbf'),
          'RF': RandomForestClassifier(n_estimators=ntrees,random_state=0)}
results = {}
y_index = []
# param_grid = {'C': [0.1, 1, 10, 100, 1000], 
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
#               'kernel': ['rbf','linear']}

# random_grid = {'bootstrap': [True, False],
#                'max_depth': [5, 7, 10, 15, 20,  None],
#                'max_features': ['auto', 'sqrt'],
#                'min_samples_leaf': [1, 2, 4, 7, 10, 15],
#                'min_samples_split': [2, 5, 10, 15],
#                'n_estimators': [200, 400, 600, 800]}

X = np.array(hof_final_train.drop(columns=['HOF']))
y = np.array(hof_final_train['HOF'])
scaler = preprocessing.MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# grid = GridSearchCV(RandomForestClassifier(), random_grid, cv=10, scoring='f1_macro',verbose = 3)
# grid.fit(X_scaled, y)

for model in models.keys():
    results[model] = {} 
    results[model]['y_predictions'] = []
    results[model]['y_pred_prob'] = []
    results[model]['precisions'] = []
    results[model]['recalls'] = []
    results[model]['f1_scores'] = []
    results[model]['auc_scores'] = []
    counter = 0    
    for train_index, test_index in skf.split(X_scaled, y):
        #print(len(train_index), len(test_index))
        if model == 'RF':
            y_index.extend(test_index)        
        x_train_fold, x_test_fold = X_scaled[train_index], X_scaled[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        clf = models[model].fit(x_train_fold, y_train_fold)
        y_probs = clf.predict_proba(x_test_fold)
        y_probs = y_probs[:, 1]
        y_pred = clf.predict(x_test_fold)
        results[model]['y_predictions'].extend(y_pred)
        results[model]['y_pred_prob'].extend(y_probs)
        precision, recall, thresholds = precision_recall_curve(y_test_fold, y_probs)
        f1, auc_score = f1_score(y_test_fold, y_pred), auc(recall, precision)
        results[model]['precisions'].append(list(np.around(precision,2)))
        results[model]['recalls'].append(list(np.around(recall,2)))
        results[model]['f1_scores'].append(np.around(f1,2))
        results[model]['auc_scores'].append(np.around(auc_score,2))
        #print(test_index)
        # if counter == 3:#'Tony Gwynn' in list(hof_final.index[test_index]):            
        #     print(counter)
        #     break
        counter+=1   

NameError: name 'hof_final_train' is not defined

In [12]:
print(fold, ' FOLD CROSS VALIDATION')
for model in models.keys():    
    print('Model: ',model)
    if model=='RF':
        print('Ntrees: ', ntrees)
    print('F1 Scores',results[model]['f1_scores'])
    print('AUC Scores', results[model]['auc_scores'])
    print('mean_f1=%.3f mean_auc=%.3f' % (np.mean(results[model]['f1_scores']), np.mean(results[model]['auc_scores'])))    
    print('************************************************')

10  FOLD CROSS VALIDATION
Model:  SVM


KeyError: 'SVM'

In [13]:
_ = tree.plot_tree(clf.estimators_[199], feature_names=hof_final_train.drop(columns=['HOF']).columns, filled=True)

NameError: name 'clf' is not defined

In [14]:
for model in models.keys():    
    hof_final_train[model+'_pred'] = 0
    for i in range(len(hof_final_train)):
        hof_final_train.loc[hof_final_train.index[i],model+'_pred'] = results[model]['y_predictions'][y_index.index(i)]
        hof_final_train.loc[hof_final_train.index[i],model+'_pred_prob'] = results[model]['y_pred_prob'][y_index.index(i)]

NameError: name 'hof_final_train' is not defined

In [15]:
hof_final_train.reset_index(inplace=True)
hof_final_train = hof_final_train.merge(df_bat[['Player', 'Player_name']].drop_duplicates('Player'), on='Player', how='left')
hof_final_train

NameError: name 'hof_final_train' is not defined

## False Negatives

In [16]:
hof_final_train[(hof_final_train['HOF']!=hof_final_train['SVM_pred'])&(hof_final_train['HOF']==1)].set_index('Player_name').sort_values('SVM_pred_prob', ascending=False)

NameError: name 'hof_final_train' is not defined

In [17]:
hof_final_train[(hof_final_train['HOF']!=hof_final_train['RF_pred'])&(hof_final_train['HOF']==1)].set_index('Player_name').sort_values('RF_pred_prob', ascending=False)

NameError: name 'hof_final_train' is not defined

## SVM and RF Predict Same for Hall of Famers

In [18]:
hof_final_train[(hof_final_train['SVM_pred']==1)  & (hof_final_train['RF_pred']==1)&(hof_final_train['HOF']==1)].set_index('Player_name').sort_values('RF_pred_prob', ascending=False)

NameError: name 'hof_final_train' is not defined

In [19]:
hof_final_train[(hof_final_train['SVM_pred']==hof_final_train['RF_pred'])&(hof_final_train['HOF']==1)].set_index('Player_name').sort_values('RF_pred_prob', ascending=False)

NameError: name 'hof_final_train' is not defined

## SVM and RF Predict Opposite for Hall of Famers

In [20]:
hof_final_train[(hof_final_train['SVM_pred']!=hof_final_train['RF_pred'])&(hof_final_train['HOF']==1)].set_index('Player_name').sort_values('RF_pred_prob', ascending=False)

NameError: name 'hof_final_train' is not defined

## False Positives

In [21]:
hof_final_train[(hof_final_train['SVM_pred']!=hof_final_train['RF_pred'])&(hof_final_train['HOF']==0)].set_index('Player_name').sort_values('RF_pred_prob', ascending=False)

NameError: name 'hof_final_train' is not defined

In [22]:
hof_final_train[(hof_final_train['SVM_pred']!=hof_final_train['HOF'])&(hof_final_train['SVM_pred']==1)].set_index('Player_name').sort_values('SVM_pred_prob', ascending=False)

NameError: name 'hof_final_train' is not defined

In [23]:
hof_final_train[(hof_final_train['RF_pred']!=hof_final_train['HOF'])&(hof_final_train['RF_pred']==1)].set_index('Player_name').sort_values('RF_pred_prob', ascending=False)

NameError: name 'hof_final_train' is not defined

In [24]:
for model in models.keys():
    if model == 'KNN':
        continue
    print(model)
    TP = len(hof_final_train[(hof_final_train[model+'_pred']==1)&(hof_final_train['HOF']==1)])
    FP = len(hof_final_train[(hof_final_train[model+'_pred']==1)&(hof_final_train['HOF']==0)])
    FN = len(hof_final_train[(hof_final_train[model+'_pred']==0)&(hof_final_train['HOF']==1)])
    precision = np.round_(TP/(TP+FP),2)
    recall = np.round(TP/(TP+FN),2)
    print('TP: ', TP, 'FP:', FP, 'FN:', FN, 'precision:', precision, 'recall:', recall)

SVM


NameError: name 'hof_final_train' is not defined

In [25]:
feature_imp = pd.Series(clf.feature_importances_,
                        index=hof_final_train.columns[1:(1+17)]).sort_values(ascending=False)
sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.show()

NameError: name 'clf' is not defined

## Running on Test Set

In [26]:
X_train = np.array(hof_final_train.drop(columns=['HOF','SVM_pred','SVM_pred_prob','RF_pred','Player_name','RF_pred_prob']).set_index('Player')) 
y_train = np.array(hof_final_train['HOF'])
X_test = np.array(hof_final_test.drop(columns=['HOF']))
y_test = np.array(hof_final_test['HOF'])
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
models = {'SVM': SVC(C=1, gamma='scale',probability=True, kernel = 'rbf'),'RF':RandomForestClassifier(n_estimators=200,random_state=0)}
test_results = {}

for model in models.keys():
    print('Model: ', model)
    clf = models[model].fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    y_probs = clf.predict_proba(X_test_scaled)
    y_probs = y_probs[:, 1]    
    hof_final_test[model+'_pred'] = y_pred
    hof_final_test[model+'_pred_prob'] = y_probs
    precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
    f1, auc_score = f1_score(y_test, y_pred), auc(recall, precision)    
    print('F1score:', f1, 'AUC:', auc_score)

NameError: name 'hof_final_train' is not defined

In [27]:
hof_final_test.reset_index(inplace=True)
hof_final_test = hof_final_test.merge(df_bat[['Player', 'Player_name']].drop_duplicates('Player'), on='Player', how='left')
hof_final_test

NameError: name 'hof_final_test' is not defined

## False Negatives

In [28]:
hof_final_test[(hof_final_test['HOF']!=hof_final_test['SVM_pred'])&(hof_final_test['HOF']==1)].set_index('Player_name').sort_values('SVM_pred_prob', ascending=False)

NameError: name 'hof_final_test' is not defined

In [29]:
hof_final_test[(hof_final_test['HOF']!=hof_final_test['RF_pred'])&(hof_final_test['HOF']==1)].set_index('Player_name').sort_values('RF_pred_prob', ascending=False)

NameError: name 'hof_final_test' is not defined

In [30]:
hof_final_test[(hof_final_test['SVM_pred']!=hof_final_test['RF_pred'])&(hof_final_test['HOF']==1)].set_index('Player_name').sort_values('RF_pred_prob', ascending=False)

NameError: name 'hof_final_test' is not defined

In [31]:
hof_final_test[(hof_final_test['SVM_pred']==1) & (hof_final_test['RF_pred']==1) & (hof_final_test['HOF']==1)].set_index('Player_name').sort_values('RF_pred_prob', ascending=False)

NameError: name 'hof_final_test' is not defined

## False Positives

In [32]:
hof_final_test[((hof_final_test['SVM_pred']==1) | (hof_final_test['RF_pred']==1))&(hof_final_test['HOF']==0)].set_index('Player_name').sort_values('RF_pred_prob', ascending=False)

NameError: name 'hof_final_test' is not defined

## Final Results

In [33]:
for model in models.keys():
    print(model)
    TP = len(hof_final_test[(hof_final_test[model+'_pred']==1)&(hof_final_test['HOF']==1)])
    FP = len(hof_final_test[(hof_final_test[model+'_pred']==1)&(hof_final_test['HOF']==0)])
    FN = len(hof_final_test[(hof_final_test[model+'_pred']==0)&(hof_final_test['HOF']==1)])
    precision = np.round_(TP/(TP+FP),2)
    recall = np.round(TP/(TP+FN),2)
    print('TP: ', TP, 'FP:', FP, 'FN:', FN, 'precision:', precision, 'recall:', recall)

SVM


NameError: name 'hof_final_test' is not defined

## Current Players

In [34]:
def data_prep(df_main, features):
    df = df_main.copy()
    col_avail = [x for x in features if x in df.columns]
    col_add = [x for x in features if x not in df.columns]
    for col in col_add:          
        if col == 'AS' or col == 'MVP':
            df[col] = df['Player'].apply(lambda x:awards(x,col,n=10))            
            continue
        info = col.split('_')
        if info[-1].isnumeric() and 'best' not in info:
            df[col]= df['Player'].apply(lambda x:rolling_max(x,int(info[-1]),info[0]))
        elif info[-1].isnumeric():
            df[col]= df['Player'].apply(lambda x:bestof(x,int(info[-1]),info[0]))
        elif info[-1] == 'BA':
            df[col] = np.where(((df['H']>3000)&(df['BA']>0.300)),1,0)
        elif info[-1] == 'ymax':
            df[col] = df['Player'].apply(lambda x:year_max(x,info[0]))        
        elif info[-1] == 'max':
            df[col] = df['Player'].apply(lambda x:col_max(x,info[0]))
    df = df[['Player']+col_add+col_avail]
    if 'HOF' in col_avail:
        df.drop(columns=['HOF'], inplace=True)
    df.set_index('Player', inplace=True)
    return df

In [35]:
df_current = career_stats(df_bat)
df_current = df_current[['Player'] + ['G', 'AB', 'R', 'H','2B','3B', 'HR','RBI', 'SB','BA', 'SLG','OPS', 'TB','Rbat','WAA','RAR','oRAR','RE24','Pos_mean','HOF','WAR','oWAR', 'Rbat+','Rfield','Rbaser','last_game']] #'BB', 'SO', 'SB'
df_current = df_current.merge(df_bat[['Player', 'Player_name']].drop_duplicates('Player'),how='left', on='Player')
df_current = df_current[df_current['last_game']>2015]
df_current.reset_index(drop=True, inplace=True)
df_current.dropna(inplace=True)
df_current

NameError: name 'career_stats' is not defined

In [36]:
# features = list(hof_final_train.drop(columns=['SVM_pred', 'SVM_pred_prob', 'RF_pred', 'RF_pred_prob', 'Player', 'Player_name']).columns)
# df_prep = data_prep(df_current, features)
# df_prep

# df_prep.to_csv('df_prep.csv')

df_prep = pd.read_csv('asset/df_prep.csv', index_col=0)
df_prep

FileNotFoundError: [Errno 2] No such file or directory: 'asset/df_prep.csv'

In [37]:
X_train = np.array(hof_final_train.drop(columns=['HOF','SVM_pred','SVM_pred_prob','RF_pred','RF_pred_prob','Player_name']).set_index('Player'))
y_train = np.array(hof_final_train['HOF'])

X_test = np.array(df_prep)
                  
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
                  
## RFC parameters from above
clf = RandomForestClassifier(n_estimators=200,random_state=0)
clf.fit(X_train_scaled,y_train)

df_prep['RF_pred'] = clf.predict(X_test_scaled)
y_probs = clf.predict_proba(X_test_scaled)

df_prep['RF_pred_prob'] = y_probs[:,1]
df_prep.reset_index(inplace=True)

df_prep = df_prep.merge(df_bat[['Player', 'Player_name']].drop_duplicates('Player'),how='left', on='Player')
df_prep

NameError: name 'hof_final_train' is not defined

In [38]:
dfFinal = df_prep.sort_values(by='RF_pred_prob',ascending=False).drop(columns=['Player'])
dfFinal = dfFinal[(dfFinal['RF_pred_prob']>=0.47)].set_index('Player_name')
dfFinal[dfFinal.columns.tolist()[11:]+dfFinal.columns.tolist()[:11]]

NameError: name 'df_prep' is not defined

## Predicting HoF Throughout a Player Career

In [39]:
dfCareer = career_stats(df_bat)
dfCareer = dfCareer[dfCareer['HOF']==1]
players = dfCareer['Player'].tolist()
df_bat2 = df_bat[df_bat['Player'].isin(players)]
df_bat2

NameError: name 'career_stats' is not defined

In [40]:
features = ['H_ymax','HR_ymax','R_ymax','RBI_ymax','BA_ymax','RAR','WAR','oWAR','WAR_best_7','WAR_7',
            'oWAR_7','oWAR_best_7','RAR_best_7','RAR_max','AS','MVP','Rfield','HOF']

In [41]:
def player_stats(main, features, clf, predType = 'binary', df = df_bat):    
    player_main = career_stats(df)[['Player']]
    player_main = player_main.merge(df[['Player', 'Player_name']].drop_duplicates('Player'),how='left', on='Player')

    col_avail = [x for x in features if x in df.columns]
    col_add = [x for x in features if x not in df.columns]
    req_extra = ['last_game','HR', 'AB', '2B', '3B', 'H','R', 'RBI', 'G','BA', 'TB']
    col_extra = [x for x in req_extra if x not in col_avail]
    season_max = df[['Player', main]].groupby(['Player']).agg('max').reset_index()
    start = min(df[main])
    if main == 'Season':
        start = 1
    for i in range(start, max(df[main])+1):
        #print(main+ ' '+str(i))
        temp_main = df[df[main]<=i]
        temp_main = temp_main[temp_main['Player'].isin(player_main['Player'])]
        temp_main = temp_main[temp_main['Player'].isin(season_max[season_max[main]>=i]['Player'])]
        temp = career_stats(temp_main)
        temp_main = temp_main[['Player'] + col_avail+col_extra+['Pos_adjust','Year','Awards']]
        temp = temp[['Player'] + col_avail+col_extra]
        temp = temp.merge(df[['Player', 'Player_name']].drop_duplicates('Player'),how='left', on='Player')
        temp.reset_index(drop=True, inplace=True)
        temp.dropna(inplace=True)    
        for col in col_add:          
            if col == 'AS' or col == 'MVP':
                temp[col] = temp['Player'].apply(lambda x:awards(x,col,df=temp_main,n=10))            
                continue
            info = col.split('_')
            if info[-1].isnumeric() and 'best' not in info:
                temp[col]= temp['Player'].apply(lambda x:rolling_max(x,int(info[-1]),info[0], df=temp_main))
            elif info[-1].isnumeric():
                temp[col]= temp['Player'].apply(lambda x:bestof(x,int(info[-1]),info[0], df=temp_main))
            elif info[-1] == 'ymax':
                temp[col] = temp['Player'].apply(lambda x:year_max(x,info[0],df = temp_main))        
            elif info[-1] == 'max':
                temp[col] = temp['Player'].apply(lambda x:col_max(x,info[0],df = temp_main))
        temp.drop(columns=col_extra, inplace=True)    
        temp.dropna(inplace=True)
        X = np.array(temp[col_add+col_avail].drop(columns=['HOF']))
        if X.shape[0]==0:
            continue
        x_scaled = scaler.transform(X)
        
        if predType == 'binary':
            y_probs = clf.predict(x_scaled)
        elif predType == 'proba':
            y_probs = clf.predict_proba(x_scaled)
            y_probs = y_probs[:, 1]
            
        temp[str(i)] = y_probs
        player_main = player_main.merge(temp[['Player',str(i)]], on='Player', how='left')
        player_main
    return player_main

# playerSeasonFullValue = player_stats('Season', features, clf, predType = 'proba', df = df_bat2)
# playerSeasonFullBinary = player_stats('Season', features, clf, predType = 'binary', df = df_bat2)
# playerSeasonFullValue.to_csv('playerSeasonFullValueHOF.csv')
# playerSeasonFullBinary.to_csv('playerSeasonFullBinaryHOF.csv')

NameError: name 'df_bat' is not defined

In [42]:
playerSeasonFullBinary = pd.read_csv('asset/playerSeasonFullBinaryHOF.csv', index_col=0)

plt.figure(figsize = (15,45))
pf = playerSeasonFullBinary.reset_index()
pf.set_index('Player_name', inplace=True)
pf.drop(['Billy Hamilton', 'Sam Thompson', 'Tommy McCarthy'], axis=0, inplace=True)
pf.drop(columns = ['index', 'Player'], axis=1, inplace=True)
ax = sns.heatmap(pf, linewidths=.5,cmap="RdBu", cbar=False, center = 0.5)

FileNotFoundError: [Errno 2] No such file or directory: 'asset/playerSeasonFullBinaryHOF.csv'

In [43]:
playerSeasonFullValue = pd.read_csv('asset/playerSeasonFullValueHOF.csv', index_col=0)

plt.figure(figsize = (15,45))
pf = playerSeasonFullValue.reset_index()
pf.set_index('Player_name', inplace=True)
pf.drop(['Billy Hamilton', 'Sam Thompson', 'Tommy McCarthy'], axis=0, inplace=True)
pf.drop(columns = ['index', 'Player'], axis=1, inplace=True)
ax = sns.heatmap(pf, linewidths=.5,cmap="RdBu", cbar=False, center = 0.5)

FileNotFoundError: [Errno 2] No such file or directory: 'asset/playerSeasonFullValueHOF.csv'

In [44]:
playerSeasonFullBinary = pd.read_csv('asset/playerSeasonFullBinaryWAR25.csv', index_col=0)

plt.figure(figsize = (15,200))
pf = playerSeasonFullBinary.reset_index()
pf.set_index('Player_name', inplace=True)
pf.drop(columns = ['index', 'Player'], axis=1, inplace=True)
ax = sns.heatmap(pf, linewidths=.5,cmap="RdBu", cbar=False, center = 0.5)

FileNotFoundError: [Errno 2] No such file or directory: 'asset/playerSeasonFullBinaryWAR25.csv'

In [45]:
playerSeasonFullValue = pd.read_csv('asset/playerSeasonFullValueWAR25.csv', index_col=0)

plt.figure(figsize = (15,200))
pf = playerSeasonFullValue.reset_index()
pf.set_index('Player_name', inplace=True)
pf.drop(columns = ['index', 'Player'], axis=1, inplace=True)
ax = sns.heatmap(pf, linewidths=.5,cmap="RdBu", cbar=False, center = 0.5)

FileNotFoundError: [Errno 2] No such file or directory: 'asset/playerSeasonFullValueWAR25.csv'